<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-разделение-данных" data-toc-modified-id="Загрузка-и-разделение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и разделение данных</a></span></li><li><span><a href="#Названия-столбцов" data-toc-modified-id="Названия-столбцов-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Названия столбцов</a></span></li><li><span><a href="#Цикл-CatBoost" data-toc-modified-id="Цикл-CatBoost-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Цикл CatBoost</a></span></li></ul></div>

## Загрузка и разделение данных

In [1]:
import pandas as pd
import pyarrow as pa
import numpy as np
import pyarrow.csv as csv
import pyarrow.parquet as pq
from tqdm import tqdm
from catboost import CatBoostClassifier
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
X_train = pq.read_table('./datasets/X_train.parquet').to_pandas()

In [4]:
y_train = pq.read_table('./datasets/y_train.parquet').to_pandas()

In [5]:
X_test = pq.read_table('./datasets/X_test.parquet').to_pandas()

## Названия столбцов

In [6]:
exhausters_X_list = ['ЭКСГАУСТЕР 4', 'ЭКСГАУСТЕР 5', 'ЭКСГАУСТЕР 6', 'ЭКСГАУСТЕР 7', 'ЭКСГАУСТЕР 8', 'ЭКСГАУСТЕР 9']
exhausters_X_list

['ЭКСГАУСТЕР 4',
 'ЭКСГАУСТЕР 5',
 'ЭКСГАУСТЕР 6',
 'ЭКСГАУСТЕР 7',
 'ЭКСГАУСТЕР 8',
 'ЭКСГАУСТЕР 9']

In [7]:
exhausters_y_list = ['ЭКСГАУСТЕР А/М №4' ,'ЭКСГАУСТЕР А/М №5', 'ЭКСГАУСТЕР А/М №6', 'ЭКСГАУСТЕР А/М №7', 'ЭКСГАУСТЕР А/М №8', 'ЭКСГАУСТЕР А/М №9']
exhausters_y_list

['ЭКСГАУСТЕР А/М №4',
 'ЭКСГАУСТЕР А/М №5',
 'ЭКСГАУСТЕР А/М №6',
 'ЭКСГАУСТЕР А/М №7',
 'ЭКСГАУСТЕР А/М №8',
 'ЭКСГАУСТЕР А/М №9']

## Цикл CatBoost

In [8]:
### exhausters cycle for M3
result = {}
for ex_X, ex_y in tqdm(zip(exhausters_X_list, exhausters_y_list)):
    X_columns = [col for col in X_train.columns if ex_X in col]
    y_columns = [col for col in y_train.columns if ex_y in col]
    features_train = X_train[X_columns]
    targets_train = y_train[y_columns]
    features_test = X_test[X_columns]
    # добавляем  .rolling(100).std(), diff(1), rolling(100).median(), row - row.mean(), row - row.median()
    for feature in features_train.columns:
        name = feature+'r100.std'
        features_train[name] = features_train[feature].rolling(100).std()
        features_test[name] = features_test[feature].rolling(100).std()
        name = feature+'diff1'
        features_train[name] = features_train[feature].diff(1)
        features_test[name] = features_test[feature].diff(1)
        name = feature+'r100.mean'
        features_train[name] = features_train[feature].rolling(100).median()
        features_test[name] = features_test[feature].rolling(100).median()
        name = feature+'-mean'
        features_train[name] = features_train[feature] - features_train[feature].mean() 
        features_test[name] = features_test[feature] - features_test[feature].mean() 
        name = feature+'-median'
        features_train[name] = features_train[feature] - features_train[feature].median()
        features_test[name] = features_test[feature] - features_test[feature].median()
    for target in targets_train.columns:
        # убираем метку 1 (останов)
        targets_train.loc[targets_train[target] == 1, target] = 0
        targets_train.loc[targets_train[target] == 2, target] = 1
        if len(targets_train[target].unique()) == 2:
            cbc = CatBoostClassifier(task_type='GPU',
                                random_seed=12345,
                                iterations=1000,
                                verbose=500,
                                auto_class_weights='Balanced')
            cbc.fit(features_train, targets_train[target])
            predictions = cbc.predict(features_test)*2
        else:
            predictions = np.array([0 for i in range(len(features_test))])
        result[target] = predictions
result = pd.DataFrame(result, index=X_test.index)

0it [00:00, ?it/s]

Learning rate set to 0.02039
0:	learn: 0.6737380	total: 73.3ms	remaining: 1m 13s
500:	learn: 0.0818767	total: 28s	remaining: 27.9s
999:	learn: 0.0537261	total: 55.6s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6650239	total: 154ms	remaining: 2m 33s
500:	learn: 0.0356350	total: 28.2s	remaining: 28.1s
999:	learn: 0.0233773	total: 56s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6580942	total: 142ms	remaining: 2m 21s
500:	learn: 0.0492714	total: 27.3s	remaining: 27.2s
999:	learn: 0.0365690	total: 54.9s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6763298	total: 158ms	remaining: 2m 37s
500:	learn: 0.1053340	total: 28.6s	remaining: 28.5s
999:	learn: 0.0683171	total: 56.4s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6522260	total: 155ms	remaining: 2m 34s
500:	learn: 0.0106526	total: 27.6s	remaining: 27.5s
999:	learn: 0.0089424	total: 49.9s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6455073	total: 159ms	remaining: 2m 38s
500:	learn: 0

1it [22:10, 1330.36s/it]

Learning rate set to 0.02039
0:	learn: 0.6671333	total: 138ms	remaining: 2m 18s
500:	learn: 0.0574536	total: 37.5s	remaining: 37.4s
999:	learn: 0.0378349	total: 1m 5s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6478348	total: 160ms	remaining: 2m 40s
500:	learn: 0.0633535	total: 28.5s	remaining: 28.3s
999:	learn: 0.0456546	total: 56.6s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6697080	total: 109ms	remaining: 1m 48s
500:	learn: 0.0773597	total: 28.1s	remaining: 28s
999:	learn: 0.0514640	total: 55.9s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6701624	total: 169ms	remaining: 2m 49s
500:	learn: 0.1220460	total: 27.5s	remaining: 27.4s
999:	learn: 0.0862705	total: 55.1s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6336078	total: 144ms	remaining: 2m 23s
500:	learn: 0.0208151	total: 27.4s	remaining: 27.3s
999:	learn: 0.0169620	total: 54.3s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6384897	total: 166ms	remaining: 2m 45s
500:	learn: 

2it [49:54, 1526.86s/it]

...

4it [1:35:49, 1450.97s/it]

Learning rate set to 0.02039
0:	learn: 0.6557781	total: 157ms	remaining: 2m 36s
500:	learn: 0.0288086	total: 37.3s	remaining: 37.2s
999:	learn: 0.0201038	total: 1m 5s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6621025	total: 153ms	remaining: 2m 32s
500:	learn: 0.0465934	total: 28.4s	remaining: 28.3s
999:	learn: 0.0321749	total: 56.6s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6746837	total: 158ms	remaining: 2m 37s
500:	learn: 0.0851613	total: 27.5s	remaining: 27.4s
999:	learn: 0.0505352	total: 55.1s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6564614	total: 176ms	remaining: 2m 56s
500:	learn: 0.0394593	total: 28.2s	remaining: 28.1s
999:	learn: 0.0251302	total: 56.1s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6548327	total: 170ms	remaining: 2m 49s
500:	learn: 0.0178179	total: 28.2s	remaining: 28.1s
999:	learn: 0.0108121	total: 55.2s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6463535	total: 192ms	remaining: 3m 12s
500:	learn

5it [2:05:01, 1559.45s/it]

Learning rate set to 0.02039
0:	learn: 0.6408871	total: 185ms	remaining: 3m 4s
500:	learn: 0.0057434	total: 40.1s	remaining: 39.9s
999:	learn: 0.0046321	total: 1m 7s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6411935	total: 219ms	remaining: 3m 38s
500:	learn: 0.0056669	total: 29.3s	remaining: 29.2s
999:	learn: 0.0046578	total: 57.2s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.5898155	total: 159ms	remaining: 2m 38s
500:	learn: 0.0000006	total: 29.8s	remaining: 29.7s
999:	learn: 0.0000002	total: 1m	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.5877060	total: 191ms	remaining: 3m 10s
500:	learn: 0.0000002	total: 30s	remaining: 29.9s
999:	learn: 0.0000001	total: 59.8s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6652239	total: 158ms	remaining: 2m 37s
500:	learn: 0.0719854	total: 28.7s	remaining: 28.6s
999:	learn: 0.0477103	total: 56.9s	remaining: 0us
Learning rate set to 0.02039
0:	learn: 0.6642379	total: 165ms	remaining: 2m 45s
500:	learn: 0.04

6it [2:27:56, 1479.39s/it]


In [ ]:
result.to_csv('y_test.csv')